# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,puerto natales,-51.7236,-72.4875,46.99,86,55,3.80,CL,1706672683
1,1,edinburgh of the seven seas,-37.0676,-12.3116,56.89,57,86,0.20,SH,1706672683
2,2,bethel,41.3712,-73.4140,34.36,77,100,8.05,US,1706672683
3,3,kovalam,8.3667,76.9967,86.09,74,75,1.14,IN,1706672684
4,4,bilibino,68.0546,166.4372,-28.70,99,100,0.36,RU,1706672684


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
humidity_map_plot = city_data_df.hvplot.points('Lng'
                                               ,'Lat'
                                               ,geo = True
                                               ,tiles = 'OSM'
                                               ,size = city_data_df['Humidity']
                                               ,frame_width = 700
                                               ,frame_height = 500
                                               ,color = 'City'
                                               ,cmap='colorwheel'
                                               ,hover_cols = ['City', 'Humidity'])


# Display the map
humidity_map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,_size,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_weather = pd.DataFrame(city_data_df[(city_data_df['Max Temp'] < 75) 
                             & (city_data_df['Humidity'] < 65)
                             & (city_data_df['Max Temp'] > 60)
                             & (city_data_df['Cloudiness'] > 40)
                            ])

# Drop any rows with null values
ideal_weather.dropna(inplace = True)

# Display sample data
ideal_weather.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
18,18,blackmans bay,-43.0167,147.3167,73.53,46,84,7.99,AU,1706672688
36,36,isiro,2.7739,27.6160,65.70,52,98,2.77,CD,1706672693
63,63,papatowai,-46.5619,169.4708,65.07,48,76,7.99,NZ,1706672703
71,71,ouadda,8.0777,22.4007,66.67,21,100,6.11,CF,1706672707
109,109,bertoua,4.5833,13.6833,67.71,34,81,0.58,CM,1706672724


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
18,blackmans bay,AU,-43.0167,147.3167,46,
36,isiro,CD,2.7739,27.6160,52,
63,papatowai,NZ,-46.5619,169.4708,48,
71,ouadda,CF,8.0777,22.4007,21,
109,bertoua,CM,4.5833,13.6833,34,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
params = {
    'categories': 'accommodation.hotel'
    ,'limit': 1
    ,'apiKey': geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row['Lat']
    lng = row['Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f'circle:{lng},{lat},{radius}'
    params["bias"] = f'proximity:{lng},{lat}'
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places?"


    # Make and API request using the params dictionaty
    name_response = requests.get(base_url, params = params)
    
    # Convert the API response to JSON format
    name_address = name_response.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
blackmans bay - nearest hotel: Villa Howden
isiro - nearest hotel: Hôtel Magbetu
papatowai - nearest hotel: No hotel found
ouadda - nearest hotel: No hotel found
bertoua - nearest hotel: Hotel Sun City
ambikapur - nearest hotel: No hotel found
invercargill - nearest hotel: The Grand
asosa - nearest hotel: Africa Hotel
essaouira - nearest hotel: Essaouira Wind Palace
yambio - nearest hotel: Kabash Inn
fortuna - nearest hotel: Comfort Inn & Suites Redwood Country
mauganj - nearest hotel: No hotel found
new norfolk - nearest hotel: Woodbridge on the Derwent
lalsot - nearest hotel: No hotel found
ormara - nearest hotel: No hotel found
fernandopolis - nearest hotel: Sotam Hotel
lagos de moreno - nearest hotel: Lagos INN
bunia - nearest hotel: Hôtel Golden Star
gorom-gorom - nearest hotel: No hotel found
al kharj - nearest hotel: شقق فندقية
gidole - nearest hotel: No hotel found
birao - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
18,blackmans bay,AU,-43.0167,147.3167,46,Villa Howden
36,isiro,CD,2.7739,27.6160,52,Hôtel Magbetu
63,papatowai,NZ,-46.5619,169.4708,48,No hotel found
71,ouadda,CF,8.0777,22.4007,21,No hotel found
109,bertoua,CM,4.5833,13.6833,34,Hotel Sun City
127,ambikapur,IN,23.1167,83.2000,38,No hotel found
139,invercargill,NZ,-46.4000,168.3500,46,The Grand
143,asosa,ET,10.0667,34.5333,29,Africa Hotel
151,essaouira,MA,31.5125,-9.7700,31,Essaouira Wind Palace
178,yambio,SS,4.5721,28.3955,34,Kabash Inn


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
hotel_map_plot = hotel_df.hvplot.points('Lng'
                                        ,'Lat'
                                        ,geo = True
                                        ,tiles = 'OSM'
                                        ,size = hotel_df['Humidity']
                                        ,frame_width = 700
                                        ,frame_height = 500
                                        ,color = 'City'
                                        ,cmap='colorwheel'
                                        ,hover_cols = ['City', 'Country', 'Humidity', 'Hotel Name'])

# Display the map
hotel_map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,_size,Country,Humidity,Hotel Name)